# Getting the raw dataset

We'll be using `twython` and `cryptocompare` as APIs to get the respective
Twitter and cryptocurrency data that will make up the project's dataset.

## Importing packages.

First, we'll import the necessary python packages.

In [1]:
# Access environment variables.
from os import environ

# Cryptocompare API
import cryptocompare

# Manipulating the raw data to save it in a ``.csv`` files.
import pandas as pd

# Loading environment variables from a `.env` file.
from dotenv import load_dotenv

# Twython API.
from twython import Twython

## Loading secrets from environment variables.

Next, we need to set credentials to access the CryptoCompare and Twython
APIs. To avoid hard coding the secrets, we'll load them from the environment,
i.e., a `.env` file.

In [2]:
# Loads environment variables from a `.env` file.
load_dotenv()

# Now the environment variables from the file are available, as if they were
# specified typically from the commandline.
TWITTER_APP_KEY = environ["TWITTER_APP_KEY"]
TWITTER_APP_SECRET = environ["TWITTER_APP_SECRET"]

## Testing the APIs.

Let's test the CryptoCompare API. `CRYPTOCOMPARE_API_KEY` should be specified
in the `.env` file so that the python package can detect it automatically as
an environment variable.

In [3]:
cryptocompare.get_price("BTC", currency="USD")

{'BTC': {'USD': 39083.25}}

Now let's test accessing Twitter's API through Twython.

In [4]:
twitter = Twython(TWITTER_APP_KEY, TWITTER_APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(
    TWITTER_APP_KEY, TWITTER_APP_SECRET, access_token=ACCESS_TOKEN
)

search_results = twitter.search(count=1, q="cryptocurrency")
print(search_results)

{'statuses': [], 'search_metadata': {'completed_in': 0.021, 'max_id': 1500587859056771072, 'max_id_str': '1500587859056771072', 'query': 'cryptocurrency', 'refresh_url': '?since_id=1500587859056771072&q=cryptocurrency&include_entities=1', 'count': 1, 'since_id': 0, 'since_id_str': '0'}}
